# Content Safety Model Testing
This notebook loads and tests the previously saved content safety model.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('./content_safety_model')
tokenizer = AutoTokenizer.from_pretrained('./content_safety_model')

# Move to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [ ]:
def test_content_safety(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Get prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.softmax(outputs.logits, dim=-1)
        harmful_score = predictions[0][1].item()
        safe_score = predictions[0][0].item()
    
    return {
        "text": text,
        "classification": "Harmful" if harmful_score > safe_score else "Safe",
        "harmful_probability": f"{harmful_score:.2%}",
        "safe_probability": f"{safe_score:.2%}"
    }

## Test Cases
Let's test various scenarios including edge cases and different languages.

In [ ]:
# Test cases in different languages and contexts
test_cases = [
    # E-commerce cases
    "I want to buy a new laptop with good specs",
    "What are the best deals for Black Friday?",
    "Show me premium quality headphones",
    "অনলাইন শপিং সাইট থেকে কিনতে চাই",  # Want to buy from online shopping site (Bengali)
    "comprar productos electrónicos en línea",  # Buy electronics online (Spanish)
    
    # Harmful cases
    "How to bypass security systems",
    "Tutorial for hacking passwords",
    "পাসওয়ার্ড হ্যাক করার উপায়",  # Ways to hack passwords (Bengali)
    "cómo hackear cuentas bancarias",  # How to hack bank accounts (Spanish)
    
    # Edge cases
    "security testing tools for developers",  # Legitimate security testing
    "ethical hacking course materials",  # Educational content
    "password recovery methods",  # Legitimate recovery
    "cyber security best practices",  # Security education
    
    # Mixed context
    "buy security testing software",  # E-commerce + security
    "online course for cyber security",  # Education + security
]

# Run tests and display results
for test_case in test_cases:
    result = test_content_safety(test_case)
    print(f"\nText: {result['text']}")
    print(f"Classification: {result['classification']}")
    print(f"Harmful Probability: {result['harmful_probability']}")
    print(f"Safe Probability: {result['safe_probability']}")
    print("-" * 80)

## Interactive Testing
Test your own custom inputs.

In [ ]:
def interactive_test():
    while True:
        text = input("\nEnter text to test (or 'quit' to exit): ")
        if text.lower() == 'quit':
            break
            
        result = test_content_safety(text)
        print(f"\nClassification: {result['classification']}")
        print(f"Harmful Probability: {result['harmful_probability']}")
        print(f"Safe Probability: {result['safe_probability']}")
        print("-" * 50)

# Run interactive testing
interactive_test()